In [4]:
#Fish
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import re
#a definition of the column names
"""    - CRIM     per capita crime rate by town
    - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
    - INDUS    proportion of non-retail business acres per town
    - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) ###note, a factor shouldn't be numeric
    - NOX      nitric oxides concentration (parts per 10 million)
    - RM       average number of rooms per dwelling
    - AGE      proportion of owner-occupied units built prior to 1940
    - DIS      weighted distances to five Boston employment centres
    - RAD      index of accessibility to radial highways                             ###another category
    - TAX      full-value property-tax rate per $10,000
    - PTRATIO  pupil-teacher ratio by town
    - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    - LSTAT    % lower status of the population
    - MEDV     Median value of owner-occupied homes in $1000's"""
    
def process_data(data):
    return(data)

data1 = pd.read_csv("housing.csv",sep="\s+")
data1.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

In [5]:
#more data processing, shows there are no null values
print(data.isnull().sum()) 

NameError: name 'data' is not defined

In [6]:
def stat_cal(df, i, split_ratio, calcor = True, drop = True):
    cori = -1
    if calcor:
        ex = df[i].mean()
        ey = df["MEDV"].mean()
        ox = np.sqrt(sum([(x - ex)**2 for x in df[i]]))
        oy = np.sqrt(sum([(y - ey)**2 for y in df["MEDV"]]))
        cori = np.sum([(x*y - ex*ey)/(ox*oy) for (x, y) in zip(df[i], df["MEDV"])])
        
    if drop:
        df = df.drop(i, axis = 1)
        
    n = int(df.shape[0]*split_ratio)
    Xtrain = df.iloc[n:].drop("MEDV", axis = 1)
    Xtrain.insert(0, 'ONE', [1]*(df.shape[0] - n))#don't include test cases
    Ytrain = df.loc[n:,"MEDV"]
    
    Xtest = df.iloc[:n,:].drop("MEDV", axis = 1)
    Xtest.insert(0, 'ONE', [1]*(n))#don't include test cases
    Ytest = df.loc[:n-1,"MEDV"]
    
    theta = np.linalg.inv(Xtrain.T.dot(Xtrain)).dot(Xtrain.T).dot(Ytrain)

    SSE = sum((Xtest.dot(theta) - Ytest)**2)
    SST = sum((Ytest - np.mean(Ytest))**2)
    R2 = 1 - SSE/SST
    R2adj = 1 - (SSE/(Xtrain.shape[0] - Xtrain.shape[1] - 1))/(SST/(Xtrain.shape[0] - 1))#adjust both SSE,SST for df
        
    
    return SSE, SST, R2, R2adj, cori
SSE, SST, R2, R2adj, cori = stat_cal(data, i, 0.2, calcor = False)

NameError: name 'data' is not defined

In [7]:
    #scrape code
    #should just make a fn
    X1 = data.iloc[100:].drop([i, "MEDV"], axis = 1)
    X1.insert(0, 'ONE', [1]*(data.shape[0] - 100))#don't include test cases
    Ytrain = data.loc[100:,"MEDV"]
    Xtest = data.iloc[:100,:].drop([i,"MEDV"], axis = 1)
    Xtest.insert(0, 'ONE', [1]*100)#don't include test cases
    Ytest = data.loc[:100,"MEDV"]
    theta = np.linalg.inv(X1.T.dot(X1)).dot(X1.T).dot(Ytrain)
    SSE = sum((Xtest.dot(theta) - Ytest)**2)
    SST = sum((Ytest - np.mean(Ytest))**2)
    R2 = 1 - SSE/SST
    R2adj = 1 - (SSE/(X1.shape[0] - X1.shape[1] - 1))/(SST/(X1.shape[0] - 1))#adjust both SSE,SST for df

NameError: name 'data' is not defined

In [8]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

data = pd.read_csv("housing.csv",sep="\s+")
data.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

corrs = data.corr()["MEDV"]
corr_indx = corrs.apply(abs).sort_values(ascending = False)[1:].index#drops the "MEDV", the y
data.plot(x = corrs.index[0], y = corrs.index[1], kind = "scatter", alpha = 0.5,
         c = data["MEDV"], cmap = plt.get_cmap("jet"), title = "best correlated  values vs. MEDV")

fig, ax = plt.subplots(nrows = 3, ncols = 4, figsize = (12,12))
fig.title = "asdf"
rw_loc = 0
col_loc = 0 

data_SSE, data_SST = stat_cal(data, i, 0.2, calcor = False, drop = False)[:2]

for i in corr_indx[:-1]:#eleven factors
    #print(rw_loc, col_loc)
    #print(ax[rw_loc, col_loc].plot(list(range(10)), list(range(10))))
    fign = ax[rw_loc, col_loc] 
    fign.scatter(data[i], data["MEDV"], alpha = 0.2)
    fign.axes.get_xaxis().set_visible(False)
    fign.axes.get_yaxis().set_visible(True)
    fign.set_ylabel('MEDV')
    fign.set_yticklabels([])
    fign.frame_on = False
    fign.set_title(i+ " vs. $$$", fontsize = 14)
    fign.text(0.5, -0.05, 'corr: ' + str("%.3f" % corrs[i]), horizontalalignment='center',
              verticalalignment='center', transform=fign.transAxes)
    line = sm.OLS(data["MEDV"],sm.add_constant(data[i])).fit()
    fign.plot(data[i], line.params[0] + line.params[1]*data[i],'r')
    #next step is to add the amount of unique variance explained by this factor
    SSE, SST, R2, R2adj, cori = stat_cal(data, i, 0.2, drop = True)
    print(SSE, SST, R2, R2adj, cori, i)
    iSSE, iSST = stat_cal(data.loc[:,[i, "MEDV"]], i, 0.2, drop = False)[:2]
    #math below is wrong, need to update
    print("Variance explained: ", cori)#variance explained by everthing(prediction on all others + noise) but this factor/total error
    print("Uniquly explain % of total explained Variance", ((data_SST - data_SSE) - SSE)/(data_SST))
    print(i, R2, R2adj, SSE, SST)
    
    col_loc += 1
    col_loc %= 4
    if col_loc == 0:
        rw_loc += 1
        
fig.suptitle('Plot of attributes vs. Home Value', fontsize=30)

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
#see how being along the chase river changes the price distribution? Graphically
#next plot pdfs of (feature, price). 3d

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
#how to calculate the pdf of this function
i = 'LSTAT'
pnts_d = [[x, y] for x,y in zip(data.loc[:,i], data.loc[:,"MEDV"])]
#print(pnts_d)
#doesn't work yet either
joint_prob, edges = np.histogramdd(pnts_d[:30], bins = 3)

ax.scatter(data[:,i], data[:,"MEDV"], zs = data["MEDV"], alpha = 0.2)

In [ ]:
np.random.randn(3, 3)[0]
#list([data.iloc[:3,1], data.iloc[:3,2]]))

In [ ]:
#how to include this in some pipeline?
#######################ONLY run once
cat_encoder = OneHotEncoder(sparse = False)
cat_var_names = ["RAD","CHAS"]
cat_vars = data1[cat_var_names]
housing_cat1hot = cat_encoder.fit_transform(cat_vars)#represents 11 binary columns of "YES/NO" for a specific radial then chase river  

cat_names = []
for i in range(len(cat_encoder.categories_)):
    for j in cat_encoder.categories_[i]:
        cat_names.append(str(cat_var_names[i]) + str(int(j)))

cats = pd.DataFrame(data = housing_cat1hot, columns = cat_names)
data1 = pd.merge(data1, cats, left_on=data1.index, right_on=cats.index,how='left')
cat_var_names.append("key_0")#gets included w/ merge
data1.drop(labels = cat_var_names, axis = 1)
print("")

In [ ]:
#should make this a pipeline
def cat_vars(lst):
    out = []
    for i in lst:
        cat_vars= data1[[i]]
        cat_encoder = OneHotEncoder(sparse = False)
        housing_cat1hot = cat_encoder.fit_transform(cat_vars)
        out.append(housing_cat1hot)
    return(out)
cat_atts = ["RAD", "CHAS"]
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([("cat", OneHotEncoder(), cat_atts)])
data2 = full_pipeline.fit_transform(data1)#just has catagorical variables? figure this out. Are the columns of arrays or many columns?
print(data1.shape)
print(data2.shape)

In [ ]:
from sklearn.model_selection import train_test_split
#do I need to keep a segment of data away from the rest perminantly?
from sklearn.linear_model import LinearRegression
y = data["MEDV"]
X = data.drop("MEDV", axis = 1)
X.insert(0,'ONE',[1]*X.shape[0])#prepend column of one's  for theta0

In [ ]:
#manually calculate the best linear regression
theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
SSE = sum((y - X.dot(theta))**2)
SST = sum((y - np.sum(y))**2)
R2 = 1 - SSE/SST
print(R2)#very good fit, but included too much

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
theta = np.linalg.inv(x_train.T.dot(x_train)).dot(x_train.T).dot(y_train)
SSE = sum((y_test - x_test.dot(theta))**2)
SST = sum((y_test - sum(y_test))**2)
R2 = 1 - SSE/SST
print("R2: ", R2)#this is >0.99999? why? regardless of how much I split my test data? need to adj R^2
MSE = SSE/(X.shape[0] - X.shape[1])#added the theta0 parameter for MEDV which was taken out
print("MSE: ", MSE)
Radj = 1 - MSE/(np.std(y_test)**2)
print("Radj", Radj)


In [ ]:
#automatically build model
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
reg = lin_reg.fit(x_train, y_train)
print("reg score: ", reg.score(x_test, y_test))#why does this perform much worse than Radj?

from sklearn.metrics import mean_squared_error
print("MSE: ", mean_squared_error(y_test, x_test.dot(reg.coef_) + reg.intercept_))
np.mean((x_test.dot(reg.coef_) + reg.intercept_ - y_test)**2)
#the reason you have to include reg.intercept_ despite having the column of 1's in X is that theta0
#in the regession = 0, as that term has no impact on the final outcome of the equation

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(x_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

forrest_scores = cross_val_score(forest_reg, x_train, y_train
                                scoring="neg_mean_squared_error", cv =10)
forest_rsme_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)